[大图](TF1.html), [下载](origin_files/TF1.ipynb)

In [1]:
import tensorflow as tf
hello=tf.constant('hello!')
sess=tf.Session()#创建一个session
hey=sess.run(hello)#使用session来运行
# sess.close()#关闭session，释放资源

# Session

In [2]:
a=tf.constant([10.1,2.0],name='a')
b=tf.constant([1.0,2.1],name='b')
sess.run(a+b)

array([ 11.10000038,   4.0999999 ], dtype=float32)

In [3]:
with tf.Session() as sess:
    sess.run(a+b)
# 无需调用sess.close()，而是运行完毕后自动释放资源

In [4]:
sess=tf.Session()
result=a+b
with sess.as_default():
    print(result.eval())

[ 11.10000038   4.0999999 ]


In [5]:
#等价于：
sess=tf.Session()
result.eval(session=sess)

array([ 11.10000038,   4.0999999 ], dtype=float32)

## InteractiveSession
注册为默认Session，以后的运算会默认在这个session中运行

In [1]:
import tensorflow as tf
sess=tf.InteractiveSession()

## ConfigProto

In [6]:
config=tf.ConfigProto(allow_soft_placement=True, # 优先使用GPU，GPU不支持的时候换成CPU，而不是报错
                     log_device_placement=True) #记录节点分配日志，生产环境下可以设定为False以减小日志量

In [7]:
sess1=tf.InteractiveSession(config=config)
sess2=tf.Session(config=config)

# 变量

In [8]:
# a=tf.matmul(x,w1)#矩阵乘法

In [9]:
import math
# 参数初始化
# 均值0，方差1的正态随机数：
weights=tf.Variable(tf.truncated_normal([3,2],stddev=1),name='weights')
#正态随机数除以神经元的个数
weights=tf.Variable(tf.truncated_normal([500,25],stddev=1/math.sqrt(500)),name='weights')

In [10]:
tf.random_normal #正态分布
tf.truncated_normal #正态分布，如果离差超过2标准差，则重新随机
tf.random_uniform#均匀分布
tf.random_gamma#Gamma分布

# 常数：
tf.zeros(shape=[1,2])
tf.ones
tf.fill(dims=(2,3),value=9)#生成一个2*3矩阵，并用9填充
tf.constant([1,2,4])

<tf.Tensor 'Const_1:0' shape=(3,) dtype=int32>

### 设定初始值

In [16]:
biases=tf.Variable(tf.zeros([3]))
w2=tf.Variable(weights.initialized_value())
w3=tf.Variable(2*weights.initialized_value())

- tf.Variable定义的tensor，一定要sess.run(w2.initializer)之后才可以参与运算

In [17]:
sess.run(w2.initializer)
sess.run(w3.initializer)
sess.run(w2+w3)

array([[ 0.0146725 , -0.01635125,  0.04498549, ..., -0.08755898,
         0.18695503,  0.08425835],
       [-0.17774236,  0.17321549, -0.04793813, ...,  0.24831538,
         0.17418024,  0.12786189],
       [ 0.01775506,  0.072007  ,  0.05958458, ...,  0.1910471 ,
         0.10692566,  0.25722879],
       ..., 
       [-0.01484126, -0.06105198, -0.00371964, ..., -0.01382964,
         0.15019251,  0.01939502],
       [-0.00563166,  0.13065287,  0.17171642, ..., -0.02934854,
         0.01903608,  0.09856674],
       [-0.17432944,  0.0522357 , -0.0521534 , ...,  0.13465658,
        -0.08157296, -0.0702959 ]], dtype=float32)

- 也可以用global_variables_initializer()初始化所有变量

In [22]:
w4=tf.Variable(weights.initialized_value())
w5=tf.Variable(weights.initialized_value())
tf.global_variables_initializer().run(session=sess)
sess.run(w4+w5)

array([[ 0.10635487, -0.10713379,  0.05419004, ...,  0.04316486,
        -0.06955445, -0.01315271],
       [ 0.0207519 ,  0.09035016, -0.07340461, ...,  0.12121008,
         0.03519265, -0.00858499],
       [-0.0608892 , -0.07099085,  0.06276519, ..., -0.01360776,
        -0.09032469,  0.00791927],
       ..., 
       [ 0.07995607, -0.07077914, -0.10500553, ..., -0.04095972,
         0.10824137, -0.08218385],
       [-0.12486154, -0.07803642,  0.07337739, ...,  0.05695054,
         0.09509261, -0.01585101],
       [-0.08684835, -0.03203673, -0.04070302, ...,  0.12899335,
        -0.04166428,  0.0677882 ]], dtype=float32)

# 一个信号传播的例子

In [12]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

# 输入信号
x=tf.constant([[0.7,0.9]])

#矩阵积
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

sess=tf.Session()
sess.run(w1.initializer)#初始化
sess.run(w2.initializer)
sess.run(y)



array([[ 3.95757794]], dtype=float32)

## placeholder占位
定义一个位置，这个位置的数据在运行时才指定，可以节省常量。

In [13]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

# 输入信号
x=tf.placeholder(tf.float32,shape=(1,2),name='input')

#矩阵积
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

sess=tf.Session()
sess.run(w1.initializer)#初始化
sess.run(w2.initializer)
sess.run(y,feed_dict={x:[[0.7,0.9]]})

array([[ 3.95757794]], dtype=float32)

# iris为例的一个完整的神经网络训练过程

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
sess=tf.Session()

- 加载数据并dummy

In [2]:
from sklearn import datasets
dataset=datasets.load_iris()
data=dataset.data
target=dataset.target
X=data
target_df=pd.Series(target)
target_df=pd.get_dummies(target_df)
target=target_df.values
from sklearn import model_selection
train_data,test_data,train_target,test_target=model_selection.train_test_split(
    data,target,test_size=0.2,train_size=0.8,random_state=12345)#划分训练集和测试集

- 基本参数

In [3]:
X=train_data
Y=train_target
dataset_size=data.shape[0]
batch_size=10

- 定义张量

In [4]:
w1=tf.Variable(tf.truncated_normal(shape=(4,10),stddev=1),name='weight1')
b1=tf.Variable(tf.truncated_normal(shape=(1,10),stddev=1),name='bias1')
w2=tf.Variable(tf.truncated_normal(shape=(10,3),stddev=1),name='weight2')
b2=tf.Variable(tf.truncated_normal(shape=(1,3),stddev=1),name='bias2')
x=tf.placeholder(shape=(None,4),dtype=tf.float32)
y=tf.placeholder(shape=(None,3),dtype=tf.float32)

- 信号正向传播

In [5]:
a=tf.nn.sigmoid(tf.matmul(x,w1)+b1)
y_hat=tf.nn.softmax(tf.matmul(a,w2)+b2)

- 误差与优化

In [6]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_hat),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cross_entropy)

- 迭代学习

In [7]:
sess.run(tf.global_variables_initializer())#全部初始化

STEPS=50000
for i in range(STEPS):
    start=(i*batch_size)%batch_size
    end=min(start+batch_size,dataset_size)
    sess.run(train_step,feed_dict={x:X[start:end],y:Y[start:end]})
    if i%5000==0:
        total_cross_entropy=sess.run(cross_entropy,feed_dict={x:X,y:Y})
        print(i,'training step,cross entropy is',total_cross_entropy)

0 training step,cross entropy is 2.11469
5000 training step,cross entropy is 0.861995
10000 training step,cross entropy is 0.6651
15000 training step,cross entropy is 0.528154
20000 training step,cross entropy is 0.436675
25000 training step,cross entropy is 0.371314
30000 training step,cross entropy is 0.322101
35000 training step,cross entropy is 0.283936
40000 training step,cross entropy is 0.253818
45000 training step,cross entropy is 0.229742


- predict

In [8]:
y_train_predict=sess.run(y_hat,feed_dict={x:train_data})
y_test_predict=sess.run(y_hat,feed_dict={x:test_data})
sess.close()

- 模型评价

In [11]:
import sklearn.metrics as metrics
print('train',metrics.confusion_matrix(train_target.argmax(axis=1),y_train_predict.argmax(axis=1)))
print('test',metrics.confusion_matrix(test_target.argmax(axis=1),y_test_predict.argmax(axis=1)))

train [[39  0  0]
 [ 0 35  3]
 [ 0  0 43]]
test [[11  0  0]
 [ 0  8  4]
 [ 0  0  7]]


In [15]:
tf.GraphKeys.GLOBAL_VARIABLES #所有变量
tf.GraphKeys.TRAINABLE_VARIABLES #所有可训练变量
tf.GraphKeys.SUMMARIES #日志相关张量
tf.GraphKeys.QUEUE_RUNNERS #处理输入的QueueRunner
tf.GraphKeys.MOVING_AVERAGE_VARIABLES #所有计算了移动平均值的变量

'moving_average_variables'

# 其它代码

In [16]:
with tf.Graph().as_default():
    pass


In [17]:
import math
# 参数初始化
# 均值0，方差1的正态随机数：
weights=tf.Variable(tf.truncated_normal([3,2],stddev=1),name='weights')
#正态随机数除以神经元的个数
weights=tf.Variable(tf.truncated_normal([500,25],stddev=1/math.sqrt(500)),name='weights')

In [18]:
# DropOut
# 每一轮训练临时性的关闭一些节点
# 降低VC维数量，减少过拟合风险

keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

NameError: name 'h_fc1' is not defined